# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn
- What is a ReAct agent
- How agents use tools autonomously
- See real compliance verification
- Watch autonomous reasoning

---


## 🤔 What is a ReAct Agent?

**ReAct** = **Rea**soning + **Act**ing

A ReAct agent:
1. 🧠 **Thinks** about the problem
2. 🔧 **Chooses** which tool to use
3. ⚡ **Executes** the tool
4. 📊 **Observes** the result
5. 🔁 **Repeats** until done

```
Question: "Check if all doors meet requirements"
     ↓
Agent: "I need to list all doors first"
     ↓
Tool: list_all_doors()
     ↓
Agent: "Now check each door's width"
     ↓
Tool: check_door_width_compliance("DOOR_001")
     ↓
Agent: "Now check the regulation"
     ↓
Tool: query_normativa("minimum door width")
     ↓
Agent: "Here's my final answer..."
```

## 🛠️ Available Tools

Our agent has 6 tools:
1. **get_room_info(room_id)** - Get room details
2. **get_door_info(door_id)** - Get door details
3. **list_all_doors()** - List all doors
4. **check_door_width_compliance(door_id)** - Check door compliance
5. **query_normativa(question)** - Query building codes
6. **calculate_egress_distance(room_id)** - Calculate evacuation distance


In [ ]:
import sys
sys.path.append('.')

from pathlib import Path
from langchain_core.messages import HumanMessage

from src.agent.graph import create_compliance_agent
from src.agent.tools import load_project_data, set_vectorstore_manager

# Optional RAG dependencies
rag_available = False
vectorstore_path = Path("vectorstore/normativa_db")
try:
    from src.rag.vectorstore_manager import VectorstoreManager
    from src.rag.qa_chain import create_qa_chain
    rag_available = vectorstore_path.exists() and any(vectorstore_path.iterdir())
except Exception:
    rag_available = False

# Load project data (use example dataset from Tutorial 1/2)
project_file = Path("data/extracted/tutorial_example.json")
load_project_data(project_file)
print(f"✅ Project data loaded: {project_file}")

# Setup RAG for query_normativa tool if available
if rag_available:
    try:
        rag = VectorstoreManager(vectorstore_path)
        rag.load_existing()
        retriever = rag.get_retriever(k=3)
        qa_chain = create_qa_chain(retriever)

        class RAGWrapper:
            def query(self, question):
                return qa_chain({"query": question})

        set_vectorstore_manager(RAGWrapper())
        print("✅ RAG system connected")
    except Exception as e:
        print("⚠️ Could not initialize RAG:", e)
        print("   Tip: Create the vectorstore before the demo.")
        print("   For setup, see docs or run: python scripts/create_vectorstore.py")
else:
    print("ℹ️ RAG vectorstore not found. Skipping CTE querying in this run.")
    print("   To enable, run: python scripts/create_vectorstore.py")

# Create agent
agent = create_compliance_agent()
print("✅ Agent created")
print("\n🤖 Ready to verify compliance!")


## 🗺️ Project Overview

We'll reuse the example dataset from previous tutorials (`data/extracted/tutorial_example.json`).

What it contains:
- Rooms with boundaries and names
- Doors with width, position, and connections
- Walls (not used directly by the agent)

We'll first preview key stats and a simple visualization to anchor the demo.


In [ ]:
# Preview basic stats and floor plan
import json
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

with open("data/extracted/tutorial_example.json", "r") as f:
    project_preview = json.load(f)

rooms = project_preview.get("rooms", [])
doors = project_preview.get("doors", [])

print("📊 Project Stats")
print(f"   Rooms: {len(rooms)}")
print(f"   Doors: {len(doors)}")

# Simple floor plan with room names and door markers
fig, ax = plt.subplots(figsize=(10, 7))

for room in rooms:
    boundary = room["boundary"]
    poly = Polygon(boundary)
    xs = [p[0] for p in boundary]
    ys = [p[1] for p in boundary]
    ax.plot(xs, ys, 'b-', linewidth=2)
    ax.fill(xs, ys, alpha=0.15, color='steelblue')
    c = poly.centroid
    ax.text(c.x, c.y, room['name'], ha='center', va='center', fontsize=9, fontweight='bold')

for door in doors:
    x, y = door.get('position', [None, None])
    if x is None:
        continue
    color = 'green' if door.get('is_egress') else 'orange'
    ax.scatter(x, y, s=120, c=color, marker='s', zorder=5)

ax.set_title('Floor Plan Preview', fontsize=13, fontweight='bold')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.grid(True, alpha=0.3)
ax.axis('equal')
plt.tight_layout()
plt.show()

print("🟦 Rooms   🟩 Egress doors   🟧 Interior doors")


## 🧠 Agent Warm-up: What tools are available?

We'll start with two quick prompts to see the agent in action, step by step:


In [ ]:
# Example 1: List all doors
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)

print(f"\n✅ Completed in {result['iterations']} iterations")


In [ ]:
# Example 2: Verify door widths
print("="*70)
print("EXAMPLE 2: Verify door widths")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


In [ ]:
# Example 1: Simple Query
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")],
    "iterations": 0
})

# Print agent's response
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)

print(f"\n✅ Completed in {result['iterations']} iterations")


In [ ]:
# Example 2: Compliance Verification
print("="*70)
print("EXAMPLE 2: Verify door widths")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

# Print agent's reasoning and answer
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


## 🎯 Summary

In this tutorial, you learned:

1. ✅ ReAct agents reason and act autonomously
2. ✅ Agents choose which tools to use
3. ✅ Can combine multiple tools in sequence
4. ✅ Provide transparent reasoning process
5. ✅ Natural language interface for complex tasks

**This completes the POC demonstration!**

## 🚀 What's Next?

The agent can now:
- **Extract** building data from CAD files
- **Calculate** geometric properties and relationships  
- **Query** building regulations using AI
- **Verify** compliance automatically
- **Reason** about complex compliance scenarios

This is the future of technical offices! 🎉


check that all of